在 Tensorflow 当中有两种途径生成变量 variable，一种是 tf.get_variable()，另一种是tf.variable()。

1. tf.name_scope()

In [1]:
import tensorflow as tf

with tf.name_scope("a_name_scope"):
    initializer = tf.constant_initializer(value=1)
    var1 = tf.get_variable(name='var1', shape=[
                           1], dtype=tf.float32, initializer=initializer)
    var2 = tf.Variable(name='var2', initial_value=[2], dtype=tf.float32)
    var21 = tf.Variable(name='var2', initial_value=[2.1], dtype=tf.float32)
    var22 = tf.Variable(name='var2', initial_value=[2.2], dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var1.name)        # var1:0
    print(sess.run(var1))   # [ 1.]
    print(var2.name)        # a_name_scope/var2:0
    print(sess.run(var2))   # [ 2.]
    print(var21.name)       # a_name_scope/var2_1:0
    print(sess.run(var21))  # [ 2.1]
    print(var22.name)       # a_name_scope/var2_2:0
    print(sess.run(var22))  # [ 2.2]

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


var1:0
[1.]
a_name_scope/var2:0
[2.]
a_name_scope/var2_1:0
[2.1]
a_name_scope/var2_2:0
[2.2]


分析：
在tf.name_scope()中使用tf.variable()定义变量的时候，虽然name都一样，但是为了不重复变量名，Tensorflow输出的变量名并不是一样的。所以，本质上var2、var21、var22并不是一样的变量。而另一方面，使用tf.get_variable()定义的变量不会被tf.name_scope()当中的名字所影响，相当于tf.name_scope()对tf.get_variable()是无效的。


2. tf.variable_scope()

In [2]:
import tensorflow as tf

with tf.variable_scope("a_variable_scope") as scope:
    initializer = tf.constant_initializer(value=3)
    var3 = tf.get_variable(name='var3', shape=[
                           1], dtype=tf.float32, initializer=initializer)
    scope.reuse_variables()
    var3_reuse = tf.get_variable(name='var3', )
    var4 = tf.Variable(name='var4', initial_value=[4], dtype=tf.float32)
    var4_reuse = tf.Variable(name='var4', initial_value=[4], dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var3.name)  # a_variable_scope/var3:0
    print(sess.run(var3))  # [ 3.]
    print(var3_reuse.name)  # a_variable_scope/var3:0
    print(sess.run(var3_reuse))  # [ 3.]
    print(var4.name)  # a_variable_scope/var4:0
    print(sess.run(var4))  # [ 4.]
    print(var4_reuse.name)  # a_variable_scope/var4_1:0
    print(sess.run(var4_reuse))  # [ 4.]

a_variable_scope/var3:0
[3.]
a_variable_scope/var3:0
[3.]
a_variable_scope/var4:0
[4.]
a_variable_scope/var4_1:0
[4.]


分析： 
上面结果中，var3和var3_reuse是同一个变量，而var4和var4_reuse是不同的变量。
在tf.variable_scope()中，使用tf.get_variable()方法创建变量时，如果这个变量已经存在，想直接使用这个变量，加上scope.reuse_variables()即可；如果没有加上scope.reuse_variables()，Tensorflow会报重复使用变量的错误。不管是在tf.name_scope还是在tf.variable_scope()中，tf.Variable()都是在创建新的变量。如果这个变量存在，则后缀会增加0、1、2等数字编号予以区别。

1. tf.Variable(<variable_name>) 和 tf.get_variable(<variable_name>)的作用与区别： tf.Variable(<variable_name>)会自动检测命名冲突并自行处理，但tf.get_variable(<variable_name>)则遇到重名的变量创建且变量名没有设置为共享变量时，则会报错。 tf.Variable(<variable_name>)和tf.get_variable(<variable_name>)都是用于在一个name_scope下面获取或创建一个变量的两种方式，区别在于： tf.Variable(<variable_name>)用于创建一个新变量，在同一个name_scope下面，可以创建相同名字的变量，底层实现会自动引入别名机制，两次调用产生了其实是两个不同的变量。 tf.get_variable(<variable_name>)用于获取一个变量，并且不受name_scope的约束。当这个变量已经存在时，则自动获取；如果不存在，则自动创建一个变量

2. tf.name_scope(<scope_name>) 与 tf.variable_scope(<scope_name>)： 
tf.name_scope(<scope_name>):主要用于管理一个图里面的各种op，返回的是一个以scope_name命名的context manager。一个graph会维护一个name_space的 
堆，每一个namespace下面可以定义各种op或者子namespace，实现一种层次化有条理的管理，避免各个op之间命名冲突。 
tf.variable_scope(<scope_name>)：一般与tf.name_scope()配合使用，用于管理一个graph中变量的名字，避免变量之间的命名冲突，tf.variable_scope(<scope_name>)允许在一个variable_scope下面共享变量。 
需要注意的是：创建一个新的variable_scope时不需要把reuse属性设置未False，只需要在使用的时候设置为True就可以了。

tf.name_scope()对tf.get_variable没有影响，但对tf.Variable有影响,tf.variable_scope对tf.get_variable有影响，会在原有的name上加上变量空间

In [13]:
#下面这两个定义是等价的
# v = tf.get_variable('v',shape=[1],initializer=tf.constant_initializer(1.0))
# v = tf.Variable(tf.constant(1.0,shape=[1]),name='v')

tf.name_scope()

In [2]:
with tf.name_scope('name_scope_test'):
    v1 = tf.get_variable('v',shape=[1],initializer=tf.constant_initializer(1.0))
    v2 = tf.Variable(tf.constant(1.0,shape=[1]),name='v')
    v3 = tf.Variable(tf.constant(1.0,shape=[1]),name='v')
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('the name of v1:', v1.name)
    print('the name of v2:', v2.name)
    print('the name of v3:', v3.name)

the name of v1: v:0
the name of v2: name_scope_test/v:0
the name of v3: name_scope_test/v_1:0


上面代码和输出结果表明，tf.name_scope()对tf.get_variable没有影响，但对tf.Variable有影响，会在原有的name上加上命名空间，也就是name_scope_test/v:0，如果在tf.name_scope作用域下有相同的name，则会自动加上命名空间和会重命名name，即name_scope_test/v:0和name_scope_test/v_1:0

In [3]:
with tf.variable_scope('variable_scope_test'):
    v1 = tf.get_variable(
        'v', shape=[1],  initializer=tf.constant_initializer(1.0))
    v2 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')
    v3 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('the name of v1:', v1.name)
    print('the name of v2:', v2.name)
    print('the name of v3:', v3.name)

the name of v1: variable_scope_test/v:0
the name of v2: variable_scope_test/v_1:0
the name of v3: variable_scope_test/v_2:0


上面代码和输出结果表明, tf.variable_scope对tf.get_variable有影响，会在原有的name上加上变量空间，也就是variable_scope_test/v:0。tf.Variable在tf.name_scope和tf.variable_scope作用域下的效果一样



In [ ]:
with tf.variable_scope('variable_scope_test'):
    v1 = tf.get_variable(
        'v', shape=[1],  initializer=tf.constant_initializer(1.0))
    v2 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')
    v3 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')

with tf.variable_scope('variable_scope_test'):
    v4 = tf.get_variable('v', shape=[1])

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('the name of v1:', v1.name)
    print('the name of v2:', v2.name)
    print('the name of v3:', v3.name)
    print('the name of v4:', v4.name)

上面的代码会报这样的错误：ValueError: Variable variable_scope_test/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope?

因为在第二块with tf.variable_scope('variable_scope_test'):处又在variable_scope_test变量命名空间下定义了name为v的变量，也就是这里（v4 = tf.get_variable('v', shape=[1])）重新定义了已存在的变量

In [ ]:
with tf.variable_scope('variable_scope_test'):
	v1 = tf.get_variable('v', shape=[1],  initializer=tf.constant_initializer(1.0))
	v2 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')
	v3 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')

with tf.variable_scope('variable_scope_test', reuse=True):
	v4 = tf.get_variable('v', shape=[1])	

with tf.Session() as sess:
	init_op = tf.global_variables_initializer()
	sess.run(init_op)
	print('the name of v1:', v1.name)
	print('the name of v2:', v2.name)
	print('the name of v3:', v3.name)
	print('the name of v4:', v4.name)
#输出为
#the name of v1: variable_scope_test/v:0
#the name of v2: variable_scope_test/v_1:0
#the name of v3: variable_scope_test/v_2:0
#the name of v4: variable_scope_test/v:0

上面的代码能正确运行，是因为在第二块with tf.variable_scope('variable_scope_test', reuse=True)处设置了reuse=True

In [ ]:
with tf.variable_scope('variable_scope_test'):
    v1 = tf.get_variable(
        'v', shape=[1],  initializer=tf.constant_initializer(1.0))
    v2 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')
    v3 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')

with tf.variable_scope('variable_scope_test', reuse=True):
    v4 = tf.get_variable('v1', shape=[1])

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('the name of v1:', v1.name)
    print('the name of v2:', v2.name)
    print('the name of v3:', v3.name)
    print('the name of v4:', v4.name)

上面的代码会报这样的错：ValueError: Variable variable_scope_test/v1 does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?

之所以报这个错，是因为设置reuse=True之后在该变量命名空间内，tf.get_variable只能获取已存在的变量而不能创建新变量

但如果又想创建变量，又想重用变量即获取变量呢？那可以用下面这个方法

In [ ]:
with tf.variable_scope('variable_scope_test'):
    v1 = tf.get_variable(
        'v', shape=[1], initializer=tf.constant_initializer(1.0))
    v2 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')
    v3 = tf.Variable(tf.constant(1.0, shape=[1]), name='v')

with tf.variable_scope('variable_scope_test') as scope:
    v4 = tf.get_variable(
        'v1', shape=[1], initializer=tf.constant_initializer(1.0))
    scope.reuse_variables()
    v5 = tf.get_variable('v', shape=[1])

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print('the name of v1:', v1.name)
    print('the name of v2:', v2.name)
    print('the name of v3:', v3.name)
    print('the name of v4:', v4.name)
    print('the name of v5:', v5.name)

# 输出为
# the name of v1: variable_scope_test/v:0
# the name of v2: variable_scope_test/v_1:0
# the name of v3: variable_scope_test/v_2:0
# the name of v4: variable_scope_test/v1:0
# the name of v5: variable_scope_test/v:0